In [ ]:
import pandas as pd
import yfinance as yf
from pandas_datareader import DataReader
from concurrent.futures import ThreadPoolExecutor


In [18]:
url = 'https://archives.nseindia.com/content/indices/ind_nifty500list.csv'
nifty_const = pd.read_csv(url)
nifty_const['Symbol'] = nifty_const['Symbol'].apply(lambda x: x + '.NS')


In [5]:
def fetch_market_cap(symbol):
    try:
        info = yf.Ticker(symbol).info
        return info.get('marketCap', None)
    except:
        return None

In [19]:
# nifty_const['MarketCap'] = nifty_const['Symbol'].apply(fetch_market_cap)
with ThreadPoolExecutor() as executor:
    market_caps = list(executor.map(fetch_market_cap, nifty_const['Symbol']))

In [20]:
nifty_const['MarketCap'] = market_caps

In [33]:
large_cap_threshold = int(20_000e7)
mid_cap_threshold = int(50_000e7)
nifty_const.sort_values('MarketCap', ascending=False)

,Company Name,Industry,Symbol,Series,ISIN Code,MarketCap
390,Reliance Industries Ltd.,Oil Gas & Consumable Fuels,RELIANCE.NS,EQ,INE002A01018,18613568274432
441,Tata Consultancy Services Ltd.,Information Technology,TCS.NS,EQ,INE467B01029,12839606550528
193,HDFC Bank Ltd.,Financial Services,HDFCBANK.NS,EQ,INE040A01034,12403824656384
212,ICICI Bank Ltd.,Financial Services,ICICIBANK.NS,EQ,INE090A01021,6750280876032
206,Hindustan Unilever Ltd.,Fast Moving Consumer Goods,HINDUNILVR.NS,EQ,INE030A01027,6292202061824
...,...,...,...,...,...,...
219,IFB Industries Ltd.,Consumer Durables,IFBIND.NS,EQ,INE559A01017,34238386176
226,Indiabulls Real Estate Ltd.,Realty,IBREALEST.NS,EQ,INE069I01010,33470175232
467,UFLEX Ltd.,Capital Goods,UFLEX.NS,EQ,INE516A01017,30982338560
303,Mahindra Logistics Ltd.,Services,MAHLOG.NS,EQ,INE766P01016,27600668672


In [40]:
def get_stocks(capitalization=None):
    if capitalization is None:
        stocks = nifty_const.Symbol.to_list() 
    elif capitalization == 'LARGECAP':
        stocks = nifty_const.sort_values('MarketCap')[-150:].Symbol.to_list()
    elif capitalization == 'MIDCAP':
        stocks = nifty_const.sort_values('MarketCap')[-300:-150].Symbol.to_list()
    elif capitalization == 'SMALLCAP':
        stocks = nifty_const.sort_values('MarketCap')[:150].Symbol.to_list()
    return stocks

In [42]:
def download_data(stocks, start_date='2015-01-01', end_date='2020-01-01'):
    data = yf.download(stocks, start=start_date, end=end_date)
    return data['Close']

In [44]:
data = download_data(get_stocks())

[*********************100%***********************]  501 of 501 completed


71 Failed downloads:
['INDIGOPNTS.NS', 'MAZDOCK.NS', 'KIMS.NS', 'GRINFRA.NS', 'FIVESTAR.NS', 'ANGELONE.NS', 'RAINBOW.NS', 'METROBRAND.NS', 'AETHER.NS', 'MAXHEALTH.NS', 'NYKAA.NS', 'CRAFTSMAN.NS', 'MANYAVAR.NS', 'CHEMPLASTS.NS', 'TMB.NS', 'JUBLINGREA.NS', 'KPIL.NS', 'MAPMYINDIA.NS', 'NSLNISP.NS', 'SUVENPHAR.NS', 'DEVYANI.NS', 'LICI.NS', 'EQUITASBNK.NS', 'DATAPATTNS.NS', 'DELHIVERY.NS', 'NAZARA.NS', 'CAMS.NS', 'SHYAMMETL.NS', 'MSUMI.NS', 'SBICARD.NS', 'BIKAJI.NS', 'MTARTECH.NS', 'ROSSARI.NS', 'LXCHEM.NS', 'ANURAS.NS', 'AWL.NS', 'RUSTOMJEE.NS', 'KALYANKJIL.NS', 'MEDPLUS.NS', 'PAYTM.NS', 'POWERINDIA.NS', 'LODHA.NS', 'GOCOLORS.NS', 'NUVOCO.NS', 'ROUTE.NS', 'ZOMATO.NS', 'LATENTVIEW.NS', 'ACI.NS', 'PVRINOX.NS', 'SUMICHEM.NS', 'MFL.NS', 'MEDANTA.NS', 'SONACOMS.NS', 'PPLPHARMA.NS', 'STARHEALTH.NS', 'RBA.NS', 'VIJAYA.NS', 'CLEAN.NS', 'GLAND.NS', 'UTIAMC.NS', 'SAPPHIRE.NS', 'HLEGLAS.NS', 'EASEMYTRIP.NS', 'CAMPUS.NS', 'HAPPSTMNDS.NS', 'IRFC.NS', 'HOMEFIRST.NS', 'APTUS.NS', 'POLICYBZR.NS', 'MANKIN

In [47]:
data.pct_change(60).shift()

,360ONE.NS,3MINDIA.NS,AARTIDRUGS.NS,AARTIIND.NS,AAVAS.NS,ABB.NS,ABBOTINDIA.NS,ABCAPITAL.NS,ABFRL.NS,ACC.NS,...,WESTLIFE.NS,WHIRLPOOL.NS,WIPRO.NS,YESBANK.NS,ZEEL.NS,ZENSARTECH.NS,ZFCVINDIA.NS,ZOMATO.NS,ZYDUSLIFE.NS,ZYDUSWELL.NS
Date,,,,,,,,,,,,,,,,,,,,,
2015-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-24,-0.243387,0.087033,0.073100,-0.045183,0.183063,-0.198335,0.220339,0.058338,0.097423,-0.111189,...,0.077319,0.165535,0.054632,-0.114184,0.056631,-0.176667,0.047374,NaN,0.031109,-0.172826
2019-12-26,-0.226271,0.078875,0.118790,-0.007254,0.167444,-0.181508,0.224498,0.028633,0.086749,-0.105928,...,0.072206,0.234026,0.056323,-0.046555,0.087036,-0.180628,0.011811,NaN,0.037535,-0.169687
2019-12-27,-0.166823,0.071516,0.135153,0.036746,0.174910,-0.128605,0.197511,0.086331,0.112740,-0.118071,...,0.100771,0.273810,0.055673,-0.047013,0.036972,-0.205957,0.024461,NaN,0.046602,-0.165801
